In [1]:
import os
import mobie
from bioio import BioImage
import bioio_tifffile
from bioio.writers import OmeTiffWriter

import shutil

In [2]:
def splitchannels(img,path):
    num_channels = len(img.channel_names)
    all = []
    allnames = []
    fname = os.path.basename(path)[:-4]

    print(f"Converting {path} to {num_channels} single channel image(s): ")
    for i in range(num_channels):
        if os.path.isfile(f"data/tmp_{fname}_C{i+1}.tif"):
            print(f"\tConverted channel {img.channel_names[i]} :D")
            all.append(f"data/tmp_{fname}_C{i+1}.tif")
            allnames.append(f"{i}_{fname}")
            continue

        try:
            tempimg = img.get_image_data("TCZYX",C=i)
            print(f"\tConverted channel {img.channel_names[i]} :D")
            OmeTiffWriter.save(tempimg, f"data/tmp_{fname}_C{i+1}.tif")
            all.append(f"data/tmp_{fname}_C{i+1}.tif")
            allnames.append(f"{i}_{fname}")
        except:
            print(f"Could not convert channel {img.channel_names[i]} :(")

    return all, allnames

In [3]:
def checkchannels(path):
    img = BioImage(path, reader=bioio_tifffile.Reader)

    if len(img.channel_names) > 1:
        return splitchannels(img,path)
    else:
        print(f"Only one channel found in {path} - no conversion needed :)")
        return [path], [os.path.basename(path)]

In [4]:
def addimg(input_path, im_name, menu_name, project_folder, dataset_name, target, max_jobs):
    unit="nanometer"
    resolution = (10, 10, 10)
    chunks = (64,64,64)
    scale_factors = 4 * [[1, 2, 2]]

    view = mobie.metadata.get_default_view("image", im_name, color="green")

    mobie.add_image(
        input_path=input_path,
        input_key='',
        root=project_folder,
        dataset_name=dataset_name,
        image_name=im_name,
        menu_name=menu_name,
        resolution=resolution,
        chunks=chunks,
        scale_factors=scale_factors,
        is_default_dataset=False,
        target=target,
        max_jobs=max_jobs,
        unit=unit,
        view=view 
    )


In [5]:
project_folder = "test_project/data"

dataset_name = "example"
dataset_folder = os.path.join(project_folder, dataset_name)

target = "local"
max_jobs = 4

In [ ]:
input_file = os.path.join("data/em_20nm_z_40_145.tif")
raw_name = "em_20nm_z_40_145"
menu_name = "em"

all_em, emnames = checkchannels(input_file)

if(len(all_em) == 1):
    addimg(all_em[0],raw_name,menu_name, project_folder, dataset_name, target, max_jobs)
    emnames = [raw_name]
else:
    for i, x in enumerate(all_em):
        addimg(x,f"{i}_{raw_name}",menu_name, project_folder, dataset_name, target, max_jobs)

In [ ]:
input_path = "data/EM04468_2_63x_pos8T_LM_raw.tif"
im_name = "EM04468_2_63x_pos8T_LM_raw"
menu_name = "lm"

all_lm, lmnames = checkchannels(input_path)

if(len(all_lm) == 1):
    addimg(all_lm[0],im_name,menu_name, project_folder, dataset_name, target, max_jobs)
else:
    for i, x in enumerate(all_lm):
        addimg(x,f"{i}_{im_name}",menu_name, project_folder, dataset_name, target, max_jobs)

In [ ]:
source_list = [emnames,lmnames]
settings = [{"color": "white", "contrastLimits": [0., 255.]}, {"color": "green", "opacity": 0.75}]

In [ ]:
mobie.create_view(dataset_folder, "default",
                  sources=source_list, display_settings=settings,
                  overwrite=True)

In [10]:
for dir in os.listdir(os.getcwd()):
    if(dir[0:3] == "tmp"):
        shutil.rmtree(dir)

for file in os.listdir("data"):
    if(file[0:3] == "tmp"):
        os.remove(os.path.join("data",file))